## Simple Recommender Model

The simple recommenders are basic systems that recommend the top items based on a certain metric or score. Here, we will build a simple recommendation model for movie database.

Dataset:
The dataset files contain data about 45,000+ movies listed.
This dataset captures some features like cast, crew, plot keywords, budget, revenue, posters,   release dates, languages, production companies, countries, TMDB vote counts, and vote averages.

In [67]:
import pandas as pd

#Load the dataset
md = pd.read_csv("movies_metadata.csv", engine='python')

# to avoid the memory errors, use the parameter engine='python' or low_memory=False

In [85]:
rows = len(md.axes[0])
cols = len(md.axes[1])
print("Number of Rows: " + str(rows))
print("Number of Columns: " + str(cols))

md.shape

In [86]:
md.head(5)

In [87]:
#Describe some details of the Dataset
perc =[.20, .40, .60, .80]
  
include =['object', 'float', 'int']

# calling describe method
md.describe(percentiles = perc, include = include)

In [88]:
md.info()

The prime parameters considered for the recommendation:

Popularity
Vote_Average
Vote_Count

In [89]:
# Convert the object type of popularity parameter into numerical 
md['popularity'] = pd.to_numeric(md['popularity'], errors = 'coerce')

# Summarize the fields:
md[["popularity","vote_average","vote_count"]].describe()

In [90]:
# Calculate the minimum number of votes required to be in the chart, min_vote above 90% quartile
min_vote = md['vote_count'].quantile(0.90)
print(min_vote)

In [91]:
mv_md = md.loc[(md.vote_count>= min_vote)]
mv_md.shape

Now, calculate the weighted ranking using:
    
    WeightedRating(WR)=(v/v+m)*R + (m/v+m)*C
    
    In the above equation,

    v is the number of votes for the movie;
    m is the minimum votes required to be listed in the chart;
    R is the average rating of the movie;
    C is the mean vote across the whole report.

In [80]:
m = min_vote
C = md['vote_average'].mean()

def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the formula
    return (v/(v+m) * R) + (m/(m+v) * C)


In [95]:
mv_md['score'] = mv_md.apply(weighted_rating, axis=1)

In [94]:
#Sort movies based on score calculated above
mv_md = mv_md.sort_values('score', ascending=False)

#Print the top 10 movies
mv_md[['title', 'vote_count', 'vote_average', 'score']].head(10)

The above list of 10 movies are recommended & will be passed to the application if any additional information needed.